In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from pprint import pprint
import time

In [2]:
import re
from datetime import datetime

def process_game_price_data(data):
    processed_data = data.copy()

    # '원가'와 '할인가'를 정수형으로 변환
    for key in ['원가', '할인가']:
        if key in processed_data and isinstance(processed_data[key], str):
            # '₩'와 ',' 제거
            cleaned_value = re.sub(r'[₩,]', '', processed_data[key])
            try:
                processed_data[key] = int(cleaned_value)
            except ValueError:
                print(f"경고: '{key}' 값을 정수로 변환할 수 없습니다: {processed_data[key]}")
                processed_data[key] = None

    # '이전과 비교한 할인가격'을 부호가 붙은 정수형으로 변환
    if '이전과 비교한 할인가격' in processed_data and isinstance(processed_data['이전과 비교한 할인가격'], str):
        original_value = processed_data['이전과 비교한 할인가격']
        sign = ''
        cleaned_value_for_sign = original_value # 부호 추출을 위한 임시 변수

        if cleaned_value_for_sign.startswith('+'):
            sign = '+'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        elif cleaned_value_for_sign.startswith('-'):
            sign = '-'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        
        # '₩'와 ',' 제거
        numeric_part = re.sub(r'[₩,]', '', cleaned_value_for_sign)
        try:
            # 부호와 숫자 부분을 합쳐 정수로 변환
            processed_data['이전과 비교한 할인가격'] = int(sign + numeric_part)
        except ValueError:
            print(f"경고: '이전과 비교한 할인가격' 값을 정수로 변환할 수 없습니다: {processed_data['이전과 비교한 할인가격']}")
            processed_data['이전과 비교한 할인가격'] = None

    # '할인 시작 날짜'를 'YYYY-MM-DD' 형식으로 변환
    if '할인 시작 날짜' in processed_data and isinstance(processed_data['할인 시작 날짜'], str):
        try:
            # 원본 날짜 형식에 맞게 파싱
            date_obj = datetime.strptime(processed_data['할인 시작 날짜'], '%d %b %Y, %H:%M')
            processed_data['할인 시작 날짜'] = date_obj.strftime('%Y-%m-%d')
        except ValueError:
            print(f"경고: '할인 시작 날짜' 형식을 변환할 수 없습니다: {processed_data['할인 시작 날짜']}")
            processed_data['할인 시작 날짜'] = None
    
    return processed_data

In [3]:
def history_log(game_name):
    history_url = f"https://isthereanydeal.com/game/{game_name}/history/"
    #print(history_url)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #Chrome을 위한 webdriver install
    driver.get(history_url)

    
    time.sleep(5)

    #div 엘리먼트 선택해서 가져오기
    game_log=list()
    div_tags = driver.find_elements(By.CSS_SELECTOR,'div.entry.svelte-17mbxpf')
    # print(len(div_tags))

    for div_tag in div_tags :
        time_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.time.svelte-17mbxpf')
        discount_date = time_tag[0].text

        store_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.svelte-zbwnbn')
        store = store_tag[0].text

        duration_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.duration.svelte-17mbxpf')
        duration = duration_tag[0].text

        cut_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.cut.svelte-17mbxpf')
        cut = cut_tag[1].text

        price_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.price.svelte-17mbxpf div')
        reg_price = price_tag[0].text
        discount_price = price_tag[2].text
        price_comp = price_tag[3].text

        #log data를 이용한 dict만들기
        log = {'할인 시작 날짜':discount_date, '플랫폼 이름': store,'할인 지속시간':duration,'할인율':cut,'원가':reg_price,'할인가':discount_price,'이전과 비교한 할인가격':price_comp}
        processed_log=process_game_price_data(log)
        # pprint(processed_log)

        game_log.append(processed_log)

    #5초로 waiting time 설정
    time.sleep(5)
    #driver 종료
    driver.quit()
    
    return game_log
    
    

In [ ]:
import os
import json

def save_json(data, game_name):
    """JSON 파일 저장 함수"""
    os.makedirs('data_json', exist_ok=True)
    filepath = f'data_json/{game_name}.json'
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
game_name = 'the-evil-within-2'
game_log = history_log(game_name)
print(game_log)
save_json(game_log,game_name)

In [ ]:
import pandas as pd
def jsonfile_to_csv(game_name):
    gamelog_df = pd.read_json(f'data_json/{game_name}.json')
    game_log.fillna('NoData',inplace = True)
    os.makedirs('data/data_csv', exist_ok=True)
    gamelog_df.to_csv(f'data_csv/{game_name}.csv', index=False)
    return

## 리팩토링 코드

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from pprint import pprint
import time
import re
from datetime import datetime
import os
import json
import pandas as pd

#원가, 할인가, 이전과 비교한 할이가격, 할인 시작 날짜를 특정 타입으로 바꿔주는 함수
def process_game_price_data(data):
    processed_data = data.copy()

    # '원가'와 '할인가'를 정수형으로 변환
    for key in ['원가', '할인가']:
        if key in processed_data and isinstance(processed_data[key], str):
            # '₩'와 ',' 제거
            cleaned_value = re.sub(r'[₩,]', '', processed_data[key])
            try:
                processed_data[key] = int(cleaned_value)
            except ValueError:
                print(f"경고: '{key}' 값을 정수로 변환할 수 없습니다: {processed_data[key]}")
                processed_data[key] = None

    # '이전과 비교한 할인가격'을 부호가 붙은 정수형으로 변환
    if '이전과 비교한 할인가격' in processed_data and isinstance(processed_data['이전과 비교한 할인가격'], str):
        original_value = processed_data['이전과 비교한 할인가격']
        sign = ''
        cleaned_value_for_sign = original_value # 부호 추출을 위한 임시 변수

        if cleaned_value_for_sign.startswith('+'):
            sign = '+'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        elif cleaned_value_for_sign.startswith('-'):
            sign = '-'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        
        # '₩'와 ',' 제거
        numeric_part = re.sub(r'[₩,]', '', cleaned_value_for_sign)
        try:
            # 부호와 숫자 부분을 합쳐 정수로 변환
            processed_data['이전과 비교한 할인가격'] = int(sign + numeric_part)
        except ValueError:
            print(f"경고: '이전과 비교한 할인가격' 값을 정수로 변환할 수 없습니다: {processed_data['이전과 비교한 할인가격']}")
            processed_data['이전과 비교한 할인가격'] = None

    # '할인 시작 날짜'를 'YYYY-MM-DD' 형식으로 변환
    if '할인 시작 날짜' in processed_data and isinstance(processed_data['할인 시작 날짜'], str):
        try:
            # 원본 날짜 형식에 맞게 파싱
            date_obj = datetime.strptime(processed_data['할인 시작 날짜'], '%d %b %Y, %H:%M')
            processed_data['할인 시작 날짜'] = date_obj.strftime('%Y-%m-%d')
        except ValueError:
            print(f"경고: '할인 시작 날짜' 형식을 변환할 수 없습니다: {processed_data['할인 시작 날짜']}")
            processed_data['할인 시작 날짜'] = None
    
    return processed_data

# 'isthereanydeal'사이트에서 특정 game의 log를 불러오는 함수
def history_log(game_name):
    history_url = f"https://isthereanydeal.com/game/{game_name}/history/"
    #print(history_url)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #Chrome을 위한 webdriver install
    driver.get(history_url)

    
    time.sleep(5)

    #div 엘리먼트 선택해서 가져오기
    game_log=list()
    div_tags = driver.find_elements(By.CSS_SELECTOR,'div.entry.svelte-17mbxpf')
    # print(len(div_tags))

    for div_tag in div_tags :
        time_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.time.svelte-17mbxpf')
        discount_date = time_tag[0].text

        store_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.svelte-zbwnbn')
        store = store_tag[0].text

        duration_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.duration.svelte-17mbxpf')
        duration = duration_tag[0].text

        cut_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.cut.svelte-17mbxpf')
        cut = cut_tag[1].text

        price_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.price.svelte-17mbxpf div')
        reg_price = price_tag[0].text
        discount_price = price_tag[2].text
        price_comp = price_tag[3].text

        #log data를 이용한 dict만들기
        log = {'할인 시작 날짜':discount_date, '플랫폼 이름': store,'할인 지속시간':duration,'할인율':cut,'원가':reg_price,'할인가':discount_price,'이전과 비교한 할인가격':price_comp}
        processed_log=process_game_price_data(log)
        # pprint(processed_log)

        game_log.append(processed_log)

    #5초로 waiting time 설정
    time.sleep(5)
    #driver 종료
    driver.quit()
    
    return game_log

# data디렉토리를 생성하고 data_json 디렉토리에 파일을 넣어주는 함수
def save_json(data, game_name):
    """JSON 파일 저장 함수"""
    os.makedirs('data/data_json', exist_ok=True)
    filepath = f'data/data_json/{game_name}.json'
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    return

def jsonfile_to_csv(game_name):
    gamelog_df = pd.read_json(f'data/data_json/{game_name}.json')
    gamelog_df.fillna('NoData',inplace = True)
    os.makedirs('data/data_csv', exist_ok=True)
    gamelog_df.to_csv(f'data/data_csv/{game_name}.csv', index=False)
    return

if __name__ == "__main__":
    game_name = 'sid-meiers-civilization-vi'
    gamelog = history_log(game_name)
    save_json(gamelog, game_name)
    jsonfile_to_csv(game_name)
    

In [7]:
data = pd.read_csv('data/data_csv/sid-meiers-civilization-vi.csv')
data.head()

,할인 시작 날짜,플랫폼 이름,할인 지속시간,할인율,원가,할인가,이전과 비교한 할인가격
0,2025-07-31,AllYouPlay,NaN,0%,83552,83552,75195
1,2025-07-31,GameBillet,NaN,17%,83552,69318,-14234
2,2025-07-31,IndieGala Store,NaN,0%,83552,83552,75209
3,2025-07-31,GameBillet,20 minutes,0%,83552,83552,76115
4,2025-07-31,Humble Store,NaN,0%,83552,83552,75209


In [ ]:
# 게임 csv파일을 받아서 gamelog를 json 및 csv 파일로 저장
csvfilename = ''
game_data = pd.read_csv('data/data_csv/sid-meiers-civilization-vi.csv')
games = data['게임 이름'].tolist()
for game_name in games :
    gamelog = history_log(game_name)
    save_json(gamelog, game_name)
    jsonfile_to_csv(game_name)

In [ ]:

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #Chrome을 위한 webdriver install
# driver.get("https://isthereanydeal.com/game/call-of-duty-modern-warfare-ii/history/")

# time.sleep(5)

# #div 엘리먼트 선택해서 가져오기
# div_tags = driver.find_elements(By.CSS_SELECTOR,'div.entry.svelte-17mbxpf')
# print(len(div_tags))

# for div_tag in div_tags :
#     time_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.time.svelte-17mbxpf')
#     discount_date = time_tag[0].text
#     print(discount_date)

#     store_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.svelte-zbwnbn')
#     store = store_tag[0].text

#     duration_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.duration.svelte-17mbxpf')
#     duration = duration_tag[0].text

#     cut_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.cut.svelte-17mbxpf')
#     cut = cut_tag[1].text

#     price_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.price.svelte-17mbxpf div')
#     reg_price = price_tag[0].text
#     discount_price = price_tag[2].text
#     price_comp = price_tag[3].text

#     log = {'할인 시작 날짜':discount_date, '플랫폼 이름': store,'할인 지속시간':duration,'할인율':cut,'원가':reg_price,'할인가':discount_price,'이전과 비교한 할인가격':price_comp}
#     pprint(log)
#     # processed_log=process_price_data_with_sign(log)

# #5초로 waiting time 설정
# time.sleep(5)
# #driver 종료
# driver.quit()

